Goal: use high-cardinality features (device_model, C14) and medium-cardinality features (C17, C20).
    
TODO:
- see one-hot encoding them just works. -> Uses too much space.
    Time is okay (31.946413278579712 on small).
    Logistic Regression has been dealing with site/app_id just fine.
    

Columns:
- id : Don't use.
- click : Using.
- hour : TODO.
- C1 : Using.
- banner_pos : Using.
- site_id : Using. (categorical + click rate)
- site_domain : Not Using. Interchangeable with site_id.
- site_category : Using. (categorical + click rate)
- app_id : Using. (categorical + click rate)
- app_domain : Not Using. Interchangeable with app_id.
- app_category : Using. (categorical + click rate)
- device_id : Using (click rate)
- device_ip : Not Using. Interchangeable with device_id.
- device_model : TODO.
- device_type : Using.
- device_conn_type : Using.
- C14 : TODO
- C15 : Using.
- C16 : Using.
- C17 : TODO
- C18 : Using.
- C19 : Using.
- C20 : TODO
- C21 : Using.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from models.base import tune_logistic_regression_pipeline, ClickRateEncoder

from tools.kaggle_tools import predict_on_test

## Stats
- Without device_model and C14:
        Tuning time:  1628.0956852436066 for 4 parameters.
        {'logistic_regression__C': 1.0},
        {0.01: -0.41000354517306076,
        0.1: -0.4057393021637744,
        1.0: -0.404804207471588,
        10.0: -0.40949817838128577},
        -0.40384372144399805

In [2]:
model_one_plus_cols = ['C1',
                'banner_pos',
                       'site_id',
                       'site_category',
                        'app_id',
                        'app_category',
                        'device_id',
                       #'device_model',
                        'device_type',
                        'device_conn_type',
                       #'C14',
                        'C15',
                        'C16',
                       'C17',
                        'C18',
                        'C19',
                       'C20',
                        'C21']

categorical_features = ['C1',
                'banner_pos',
                       'site_id',
                       'site_category',
                        'app_id',
                        'app_category',
                        'device_id',
                       #'device_model',
                        'device_type',
                        'device_conn_type',
                        #'C14',
                        'C15',
                        'C16',
                       'C17',
                        'C18',
                        'C19',
                       'C20',
                        'C21']

In [3]:
oh_encoder = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer([
    ('one_hot_encoding', oh_encoder, categorical_features)
])

lg = LogisticRegression(solver='liblinear')
pipeline = Pipeline([
                ('preprocessing', preprocessor),
                 ('logistic_regression', lg)])

In [4]:
df_train = pd.read_csv('data/train_small.csv')

In [5]:
params = np.logspace(-2, 1, num=4)

In [6]:
best_C, params_dict_ls, scores, test_score = \
    tune_logistic_regression_pipeline(df_train, pipeline, params)

Tuning time:  1628.0956852436066


In [7]:
best_C, dict(zip(params, scores)), test_score

({'logistic_regression__C': 1.0},
 {0.01: -0.41000354517306076,
  0.1: -0.4057393021637744,
  1.0: -0.404804207471588,
  10.0: -0.40949817838128577},
 -0.40384372144399805)

In [8]:
raise

RuntimeError: No active exception to reraise

In [ ]:
df_test = pd.read_csv('data/test_tiny.csv', dtype={'id': 'uint64'})
df_test.id.dtype

In [ ]:
clicks = df_train.click
#param = {'logistic_regression__C': 0.021544346900318832}
predict_on_test(df_train, clicks, pipeline, best_C, df_test, fname=None)